In [1]:
import numpy as np
import astropy.units as u
from astropy.io import fits
import time 
from importlib import reload
import copy
from datetime import datetime
today = int(datetime.today().strftime('%Y%m%d'))
from IPython.display import clear_output

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.patches import Circle, Rectangle

import scoobi
from scoobi.math_module import xp, _scipy, ensure_np_array
from scoobi.imshows import imshow1, imshow2, imshow3
import scoobi.scoobi as scoobi
import scoobi.utils as utils

import lina

print(scoobi.__file__)
print(lina.__file__)

import purepyindi
from purepyindi import INDIClient
import purepyindi2
from purepyindi2 import IndiClient

client0 = INDIClient('localhost', 7624)
client0.start()

client = IndiClient()
client.connect()
client.get_properties()

def restart_clients():
    client0 = INDIClient('localhost', 7624)
    client0.start()

    client = IndiClient()
    client.connect()
    client.get_properties()

/opt/conda/envs/km310env/lib/python3.10/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


/home/kianmilani/Projects/scoob-interface/scoobi/scoobi.py
/home/kianmilani/Projects/lina/lina/__init__.py


In [14]:
reload(scoobi)
I = scoobi.SCOOBI(scicam_channel='camsci', dm_channel='dm00disp03')
I.npsf = 256

In [5]:
reload(scoobi)
xc, yc = (4600, 3400)
xc, yc = (4500, 3300)
npsf = 256
scoobi.set_zwo_roi(xc, yc, npsf, client0)


In [ ]:
I.set_zwo_exp_time(0.0005, client0)

Set the ZWO exposure time to 5.00e-04s


In [ ]:
I.set_zwo_gain(120, client0)

Set the ZWO gain setting to 120.0


In [ ]:
I.set_fib_atten(34, client)

Set the fiber attenuation to 34.0


In [ ]:
scoobi.move_psf(50, 0, client0)

In [ ]:
scoobi.move_psf(0, -50, client0)

# Run FDPR2

In [ ]:
from magpyx.utils import ImageStream
fdpr2_amp_stream = ImageStream('fdpr2_amp')
fdpr2_phs_stream = ImageStream('fdpr2_phase')

In [ ]:
reload(lina.utils)

ring_13 = lina.utils.make_ring(rad=13, Nact=34)
ring_14 = lina.utils.make_ring(rad=14, Nact=34)
ring_15 = lina.utils.make_ring(rad=15, Nact=34)
ring_16 = lina.utils.make_ring(rad=16, Nact=34)
imshow3(ring_14, ring_15, ring_16)

cross = lina.utils.make_cross_command(xc=[-1, 3], yc=[2,-2])

poke = np.zeros((34,34))
poke[17,17] = 1
# poke[25,17] = 1
# poke[26,19] = 1
imshow2(cross, poke)

In [ ]:
amp = 20e-9
command = amp*ring_15

I.add_dm(command)

In [ ]:
from magpyx.fdpr2 import console, tools

config_params = tools.Configuration('fdpr2_fpm')
# config_params = tools.Configuration('fdpr2_kilo_dmdiv_km')

fitdict, Imeas_cen = console.estimate_oneshot(config_params)
I.zero_dm()

fitdict.update({
    'AMP':fdpr2_amp_stream.grab_latest(),
    'PHS':fdpr2_phs_stream.grab_latest(),
    'dm_command':command,
    'Imeas':Imeas_cen, 
})

imshow2(fitdict['AMP'], fitdict['PHS'], npix=100)

In [ ]:
utils.save_pickle('fdpr2_ref.pkl', fitdict)